In [ ]:
import os
import random
import shutil
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import Augmentor
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, Input

# ✅ Step 1: Mount Google Drive and Set Dataset Paths
from google.colab import drive

drive.mount('/content/drive')

BASE_DIR = '/content/drive/MyDrive/CNN Project/Skin cancer ISIC The International Skin Imaging Collaboration'
TRAIN_DIR = os.path.join(BASE_DIR, "Train")
VAL_DIR = os.path.join(BASE_DIR, "Val")

# ✅ Step 2: Check if Dataset Exists
if not os.path.exists(TRAIN_DIR):
    raise FileNotFoundError(f"🚨 Training directory not found: {TRAIN_DIR}. Please check the path!")

if not os.path.exists(VAL_DIR):
    os.makedirs(VAL_DIR, exist_ok=True)
    for class_folder in os.listdir(TRAIN_DIR):
        class_path = os.path.join(TRAIN_DIR, class_folder)
        if os.path.isdir(class_path):
            images = os.listdir(class_path)
            random.shuffle(images)
            val_count = int(len(images) * 0.2)
            val_class_path = os.path.join(VAL_DIR, class_folder)
            os.makedirs(val_class_path, exist_ok=True)
            for img in images[:val_count]:
                shutil.move(os.path.join(class_path, img), os.path.join(val_class_path, img))
    print("✅ Validation dataset successfully created.")

# ✅ Step 3: Define Image Parameters
IMG_SIZE = (180, 180)
BATCH_SIZE = 32

# ✅ Step 4: Data Preprocessing
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(TRAIN_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')
val_generator = val_datagen.flow_from_directory(VAL_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')

# ✅ Step 5: Dataset Visualization
def show_images(generator):
    class_labels = list(generator.class_indices.keys())
    fig, axes = plt.subplots(3, 3, figsize=(10, 10))
    for i in range(9):
        img, label = next(generator)
        ax = axes[i // 3, i % 3]
        ax.imshow(img[0])
        ax.set_title(class_labels[np.argmax(label[0])])
        ax.axis("off")
    plt.show()

show_images(train_generator)

# ✅ Step 6: Build Custom CNN Model
model = models.Sequential([
    Input(shape=(180, 180, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(9, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# ✅ Step 7: Train Initial Model
history = model.fit(train_generator, validation_data=val_generator, epochs=20)

# ✅ Step 8: Plot Training Performance
def plot_performance(history):
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title("Loss over epochs")
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Val Accuracy')
    plt.title("Accuracy over epochs")
    plt.legend()
    plt.show()

plot_performance(history)

# ✅ Step 9: Handle Class Imbalance Using Augmentor
def augment_data(dataset_path):
    for class_folder in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_folder)
        if os.path.isdir(class_path):
            p = Augmentor.Pipeline(class_path)
            p.flip_left_right(probability=0.5)
            p.rotate(probability=0.5, max_left_rotation=10, max_right_rotation=10)
            p.zoom(probability=0.3, min_factor=1.1, max_factor=1.5)
            p.sample(500)
            print(f"✅ Augmented class: {class_folder}")

augment_data(TRAIN_DIR)

# ✅ Step 10: Train Model on Augmented Data
train_generator = train_datagen.flow_from_directory(TRAIN_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')

history_augmented = model.fit(train_generator, validation_data=val_generator, epochs=30)

plot_performance(history_augmented)

# ✅ Step 11: Save Model
model.save("melanoma_cnn_model.h5")
print("✅ Model training complete. Model saved as melanoma_cnn_model.h5")
